<a href="https://colab.research.google.com/github/fabiobento/dnn-course-2024-1/blob/main/00_course_folder/hp_tuning/02%20-%20hp_getting_started.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Adaptado de [Getting started with KerasTuner](https://keras.io/guides/keras_tuner/getting_started/) de Luca Invernizzi, James Long, Francois Chollet e Tom O'Malley, Haifeng Jin em  [Keras Developer Guides](https://keras.io/guides/).

# Primeiros passos com o KerasTuner

In [29]:
pip install -q -U keras-tuner

Note: you may need to restart the kernel to use updated packages.


## Introdução

O KerasTuner é uma biblioteca de ajuste de hiperparâmetros.

Ela tem uma forte integração com os fluxos de trabalho do Keras, mas não se limita a eles: você pode usá-la para você pode usá-la para ajustar modelos do scikit-learn ou qualquer outro _framework_.

Neste tutorial, você verá como ajustar a arquitetura do modelo, o processo de treinamento e as etapas de preprocessamento de dados com o KerasTuner. Vamos começar com um exemplo simples.

## Ajustar a arquitetura do modelo

A primeira coisa que precisamos fazer é escrever uma função que retorne um modelo compilado do
modelo do Keras.

Ela recebe um argumento `hp` para definir os hiperparâmetros durante a construção do modelo.

### Definir o espaço de pesquisa

No exemplo de código a seguir, definimos um modelo do Keras com duas camadas `Dense`.

Queremos ajustar o número de unidades na primeira camada `Dense`.

Para isso, apenas definimos um hiperparâmetro inteiro com `hp.Int('units', min_value=32, max_value=512, step=32)`, cujo intervalo é de 32 a 512, inclusive.

Ao fazer a amostragem a partir dele, a etapa mínima para percorrer o intervalo é 32.

In [30]:
import tensorflow as tf
from tensorflow.keras import layers


def build_model(hp):
    model = tf.keras.Sequential()
    model.add(layers.Flatten())
    model.add(
        layers.Dense(
            # Definir o hiperparâmetro.
            units=hp.Int("units", min_value=32, max_value=512, step=32),
            activation="relu",
        )
    )
    model.add(layers.Dense(10, activation="softmax"))
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model


Você pode testar rapidamente se o modelo foi criado com sucesso.

In [31]:
import keras_tuner

build_model(keras_tuner.HyperParameters())

<Sequential name=sequential_5, built=False>

Há também muitos outros tipos de hiperparâmetros.

Podemos definir vários hiperparâmetros na função.

No código a seguir, ajustamos se devemos:
* usar uma camada `Dropout` com `hp.Boolean()`
* ajustamos a função de ativação a ser usar com `hp.Choice()`
* ajustamos a taxa de aprendizado do otimizador com `hp.Float()`.

In [32]:

def build_model(hp):
    model = tf.keras.Sequential()
    model.add(layers.Flatten())
    model.add(
        layers.Dense(
            # Ajuste o número de unidades.
            units=hp.Int("units", min_value=32, max_value=512, step=32),
            # Ajuste a função de ativação a ser usada.
            activation=hp.Choice("activation", ["relu", "tanh"]),
        )
    )
    # Ajuste de dropout
    if hp.Boolean("dropout"):
        model.add(layers.Dropout(rate=0.25))
    model.add(layers.Dense(10, activation="softmax"))
    # Defina a taxa de aprendizado do otimizador como um hiperparâmetro.
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model


build_model(keras_tuner.HyperParameters())

<Sequential name=sequential_6, built=False>

Conforme mostrado abaixo, os hiperparâmetros são valores reais.

De fato, eles são apenas funções que retornam valores reais.

Por exemplo, `hp.Int()` retorna um valor `int`.

Portanto, você pode colocá-los em variáveis, loops for ou condições if.

In [33]:
hp = keras_tuner.HyperParameters()
print(hp.Int("units", min_value=32, max_value=512, step=32))

32


Você também pode definir os hiperparâmetros antecipadamente e manter o código do Keras em
em uma função separada.

In [34]:

def call_existing_code(units, activation, dropout, lr):
    model = tf.keras.Sequential()
    model.add(layers.Flatten())
    model.add(layers.Dense(units=units, activation=activation))
    if dropout:
        model.add(layers.Dropout(rate=0.25))
    model.add(layers.Dense(10, activation="softmax"))
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model


def build_model(hp):
    units = hp.Int("units", min_value=32, max_value=512, step=32)
    activation = hp.Choice("activation", ["relu", "tanh"])
    dropout = hp.Boolean("dropout")
    lr = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    # Chame o código de construção de modelo existente com os valores de hiperparâmetro.
    model = call_existing_code(
        units=units, activation=activation, dropout=dropout, lr=lr
    )
    return model


build_model(keras_tuner.HyperParameters())

<Sequential name=sequential_7, built=False>

Cada um dos hiperparâmetros é identificado exclusivamente por seu nome (o primeiro argumento).

Para ajustar o número de unidades em diferentes camadas `Dense` separadamente como hiperparâmetros diferentes, damos a eles nomes diferentes como `f "units_{i}"`.

Notavelmente, esse também é um exemplo de criação de hiperparâmetros condicionais.

Há muitos hiperparâmetros que especificam o número de unidades nas camadas `Dense` densas.
> O número de tais hiperparâmetros é decidido pelo número de camadas, que também é um hiperparâmetro.

Portanto, o número total de hiperparâmetros usado pode ser diferente de uma tentativa para outra.

Alguns hiperparâmetros são usados somente quando uma determinada condição é satisfeita.
> Por exemplo, `units_3` só é usado quando `num_layers` for maior que 3.

Com o KerasTuner, você pode definir facilmente esses hiperparâmetros dinamicamente durante a criação do modelo.

In [35]:

def build_model(hp):
    model = tf.keras.Sequential()
    model.add(layers.Flatten())
   # Ajuste o número de camadas.
    for i in range(hp.Int("num_layers", 1, 3)):
        model.add(
            layers.Dense(
                # Ajuste o número de unidades separadamente.
                units=hp.Int(f"units_{i}", min_value=32, max_value=512, step=32),
                activation=hp.Choice("activation", ["relu", "tanh"]),
            )
        )
    if hp.Boolean("dropout"):
        model.add(layers.Dropout(rate=0.25))
    model.add(layers.Dense(10, activation="softmax"))
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model


build_model(keras_tuner.HyperParameters())

<Sequential name=sequential_8, built=False>

### Iniciar a busca

Depois de definir o espaço de busca, precisamos selecionar uma classe de sintonizador para executar a pesquisa.

Você pode escolher entre `RandomSearch`, `BayesianOptimization` e `Hyperband`, que correspondem a diferentes algoritmos de ajuste.
> Aqui usamos o `RandomSearch` como exemplo.

Para inicializar o ajustador, precisamos especificar vários argumentos no inicializador.

* `hypermodel`. A função de construção de modelo, que é `build_model` em nosso caso.
* `objective`. O nome do objetivo a ser otimizado (onde minimizar ou maximizar é automaticamente inferido para métricas incorporadas). Apresentaremos como como usar métricas personalizadas mais adiante neste tutorial.
* `max_trials`. O número total de tentativas a serem executadas durante a pesquisa.
* `executions_per_trial`. O número de modelos que devem ser criados e ajustados para cada tentativa. Diferentes tentativas têm diferentes valores de hiperparâmetro. As execuções dentro da mesma tentativa têm os mesmos valores de hiperparâmetro. O objetivo de ter várias execuções por tentativa é reduzir a variação dos resultados e, portanto, ser capaz de avaliar com mais precisão o desempenho de um modelo. Se quiser obter resultados mais rapidamente, você poderá definir `executions_per_trial=1` (rodada única de treinamento para cada configuração de modelo).
* `overwrite`. Controla se os resultados anteriores devem ser substituídos no mesmo diretório ou, em vez disso, retomar a pesquisa anterior. Aqui, definimos `overwrite=True` para iniciar uma nova pesquisa e ignorar os resultados anteriores.
* `directory`. Um caminho para um diretório para armazenar os resultados da pesquisa.
* `project_name`. O nome do subdiretório no `directory`.

In [36]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=2,
    executions_per_trial=3,
    overwrite=True,
    directory="my_dir",
    project_name="helloworld",
)

Você pode imprimir um resumo do espaço de pesquisa:

In [37]:
tuner.search_space_summary()

Search space summary
Default search space size: 5
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
dropout (Boolean)
{'default': False, 'conditions': []}
lr (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


Antes de iniciar a pesquisa, vamos preparar o conjunto de dados MNIST.

In [38]:
import numpy as np

(x, y), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x[:-10000]
x_val = x[-10000:]
y_train = y[:-10000]
y_val = y[-10000:]

x_train = np.expand_dims(x_train, -1).astype("float32") / 255.0
x_val = np.expand_dims(x_val, -1).astype("float32") / 255.0
x_test = np.expand_dims(x_test, -1).astype("float32") / 255.0

num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

Em seguida, inicie a busca pela melhor configuração de hiperparâmetros.

Todos os argumentos passados para `search` são passados para `model.fit()` em cada execução. Lembre-se de passar `validation_data` para avaliar o modelo.

In [39]:
tuner.search(x_train, y_train, epochs=2, validation_data=(x_val, y_val))

Trial 2 Complete [00h 00m 13s]
val_accuracy: 0.9688666661580404

Best val_accuracy So Far: 0.9688666661580404
Total elapsed time: 00h 00m 30s


Durante a `busca`, a função de criação de modelos é chamada com diferentes valores de hiperparâmetro em diferentes tentativas.

Em cada tentativa:
* o sintonizador gera um novo conjunto de valores de hiperparâmetros para construir o modelo.
* O modelo é então treinado e avaliado.
* As métricas são registradas.

Assim o tuner progressivamente explora progressivamente o espaço e finalmente encontra um bom conjunto de valores de hiperparâmetros.

### Consultar os resultados

Quando a pesquisa terminar, você poderá recuperar o(s) melhor(es) modelo(s).
> O modelo é salvo em sua época de melhor desempenho avaliada em `validation_data`.

In [40]:
# Obtenha os 2 principais modelos.
models = tuner.get_best_models(num_models=2)
best_model = models[0]

best_model.build(input_shape=(None, 28, 28, 1))
best_model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 96)             │        75,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 96)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           970 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 76,330 (298.16 KB)

 Trainable params: 76,330 (298.16 KB)

 Non-trainable params: 0 (0.00 B)

Você também pode imprimir um resumo dos resultados da pesquisa.

In [41]:
tuner.results_summary()

Results summary
Results in my_dir/helloworld
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 1 summary
Hyperparameters:
num_layers: 1
units_0: 96
activation: relu
dropout: True
lr: 0.002752738682669076
units_1: 384
units_2: 32
Score: 0.9688666661580404

Trial 0 summary
Hyperparameters:
num_layers: 3
units_0: 352
activation: relu
dropout: False
lr: 0.00036999076091727033
units_1: 32
units_2: 32
Score: 0.9679666757583618


Você encontrará logs detalhados, pontos de verificação, etc., na pasta `my_dir/helloworld`, ou seja, `directory/project_name`.

Você também pode visualizar os resultados de ajuste usando o TensorBoard e o plug-in HParams.
Para obter mais informações, acesse [_Visualize the hyperparameter tuning process_](https://keras.io/guides/keras_tuner/visualize_tuning/).

### Treinar novamente o modelo

Se quiser treinar o modelo com todo o conjunto de dados, você poderá recuperar os
melhores hiperparâmetros e treinar novamente o modelo por conta própria.

In [42]:
# Obtenha os 2 principais hiperparâmetros.
best_hps = tuner.get_best_hyperparameters(5)
# Crie o modelo com o melhor hp.
model = build_model(best_hps[0])
# Treinamento com todo o conjunto de dados.
x_all = np.concatenate((x_train, x_val))
y_all = np.concatenate((y_train, y_val))
model.fit(x=x_all, y=y_all, epochs=1)

1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 931us/step - accuracy: 0.8693 - loss: 0.4294


## Ajuste o treinamento do modelo

Para ajustar o processo de construção do modelo, precisamos criar uma subclasse da classe `HyperModel`, que também facilita o compartilhamento e a reutilização de hipermodelos.

Precisamos substituir `HyperModel.build()` e `HyperModel.fit()` para ajustar o processo de construção e treinamento do modelo, respectivamente.
> O método `HyperModel.build()` é o mesmo que a função de construção de modelo, que cria um modelo do Keras usando os hiperparâmetros e o retorna.

Em `HyperModel.fit()`, você pode acessar o modelo retornado por `HyperModel.build()`, `hp` e todos os argumentos passados para `search()`. Você precisa treinar o modelo e retornar o histórico de treinamento.

No código a seguir, ajustaremos o argumento `shuffle` em `model.fit()`.

Geralmente, não é necessário ajustar o número de épocas porque um _callback_ é passado para `model.fit()` para salvar o modelo em sua melhor época avaliada por `validation_data`.

> **Nota**: Os `**kwargs` devem sempre ser passados para `model.fit()` porque ele contém os retornos de chamada para salvar o modelo e os plug-ins do tensorboard.

In [43]:

class MyHyperModel(keras_tuner.HyperModel):
    def build(self, hp):
        model = tf.keras.Sequential()
        model.add(layers.Flatten())
        model.add(
            layers.Dense(
                units=hp.Int("units", min_value=32, max_value=512, step=32),
                activation="relu",
            )
        )
        model.add(layers.Dense(10, activation="softmax"))
        model.compile(
            optimizer="adam",
            loss="categorical_crossentropy",
            metrics=["accuracy"],
        )
        return model

    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            # Ajuste se os dados devem ser embaralhados em cada época.
            shuffle=hp.Boolean("shuffle"),
            **kwargs,
        )


Novamente, podemos fazer uma verificação rápida para ver se o código funciona corretamente.

In [44]:
hp = keras_tuner.HyperParameters()
hypermodel = MyHyperModel()
model = hypermodel.build(hp)
hypermodel.fit(hp, model, np.random.rand(100, 28, 28), np.random.rand(100, 10))

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1211 - loss: 13.0318  


## Ajuste o pré-processamento de dados

Para ajustar o pré-processamento de dados, basta adicionar uma etapa adicional em `HyperModel.fit()`, em que podemos acessar o conjunto de dados a partir dos argumentos.

No código a seguir, ajustamos se devemos normalizar os dados antes de treinar o modelo.

Desta vez, colocamos explicitamente `x` e `y` na assinatura da função porque precisamos usá-los.

In [45]:

class MyHyperModel(keras_tuner.HyperModel):
    def build(self, hp):
        model = tf.keras.Sequential()
        model.add(layers.Flatten())
        model.add(
            layers.Dense(
                units=hp.Int("units", min_value=32, max_value=512, step=32),
                activation="relu",
            )
        )
        model.add(layers.Dense(10, activation="softmax"))
        model.compile(
            optimizer="adam",
            loss="categorical_crossentropy",
            metrics=["accuracy"],
        )
        return model

    def fit(self, hp, model, x, y, **kwargs):
        if hp.Boolean("normalize"):
            x = layers.Normalization()(x)
        return model.fit(
            x,
            y,
            # Tune whether to shuffle the data in each epoch.
            shuffle=hp.Boolean("shuffle"),
            **kwargs,
        )


hp = keras_tuner.HyperParameters()
hypermodel = MyHyperModel()
model = hypermodel.build(hp)
hypermodel.fit(hp, model, np.random.rand(100, 28, 28), np.random.rand(100, 10))

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0983 - loss: 11.5941  


Se um hiperparâmetro for usado tanto em `build()` quanto em `fit()`, você poderá defini-lo em `build()` e usar `hp.get(hp_name)` para recuperá-lo em `fit()`.

Usamos o tamanho da imagem como exemplo.
> Ele é usado como a forma de entrada em `build()` e usado pela etapa de pré-processamento de dados para cortar as imagens em `fit()`.

In [46]:

class MyHyperModel(keras_tuner.HyperModel):
    def build(self, hp):
        image_size = hp.Int("image_size", 10, 28)
        inputs = tf.keras.Input(shape=(image_size, image_size))
        outputs = layers.Flatten()(inputs)
        outputs = layers.Dense(
            units=hp.Int("units", min_value=32, max_value=512, step=32),
            activation="relu",
        )(outputs)
        outputs = layers.Dense(10, activation="softmax")(outputs)
        model = tf.keras.Model(inputs, outputs)
        model.compile(
            optimizer="adam",
            loss="categorical_crossentropy",
            metrics=["accuracy"],
        )
        return model

    def fit(self, hp, model, x, y, validation_data=None, **kwargs):
        if hp.Boolean("normalize"):
            x = layers.Normalization()(x)
        image_size = hp.get("image_size")
        cropped_x = x[:, :image_size, :image_size, :]
        if validation_data:
            x_val, y_val = validation_data
            cropped_x_val = x_val[:, :image_size, :image_size, :]
            validation_data = (cropped_x_val, y_val)
        return model.fit(
            cropped_x,
            y,
            # Tune whether to shuffle the data in each epoch.
            shuffle=hp.Boolean("shuffle"),
            validation_data=validation_data,
            **kwargs,
        )


tuner = keras_tuner.RandomSearch(
    MyHyperModel(),
    objective="val_accuracy",
    max_trials=3,
    overwrite=True,
    directory="my_dir",
    project_name="tune_hypermodel",
)

tuner.search(x_train, y_train, epochs=2, validation_data=(x_val, y_val))

Trial 3 Complete [00h 00m 03s]
val_accuracy: 0.9606000185012817

Best val_accuracy So Far: 0.9606000185012817
Total elapsed time: 00h 00m 11s


### Treinar novamente o modelo

O uso do `HyperModel` também permite que você treine novamente o melhor modelo por conta própria.

In [47]:
hypermodel = MyHyperModel()
best_hp = tuner.get_best_hyperparameters()[0]
model = hypermodel.build(best_hp)
hypermodel.fit(best_hp, model, x_all, y_all, epochs=1)

1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 559us/step - accuracy: 0.8523 - loss: 0.5241


## Especifique o objetivo de ajuste

Em todos os exemplos anteriores, usamos apenas a acurácia de validação (`"val_accuracy"`) como o objetivo de ajuste para selecionar o melhor modelo. Na verdade, você pode usar qualquer métrica como objetivo. A métrica mais comumente usada é `"val_loss"`, que é a perda de validação.

### Métrica integrada como objetivo

Há muitas outras métricas incorporadas no Keras que você pode usar como objetivo. Aqui está [uma lista das métricas incorporadas](https://keras.io/api/metrics/).

Para usar uma métrica integrada como objetivo, você precisa seguir estas etapas:

* Compilar o modelo com a métrica incorporada. Por exemplo, você deseja usar `MeanAbsoluteError()`. Você precisa compilar o modelo com `metrics=[MeanAbsoluteError()]`. Em vez disso, você também pode usar sua _string_: `metrics=["mean_absolute_error"]`.

* Identificar a _string_ do nome do objetivo. A _string_ do nome do objetivo está sempre no formato de `f "val_{metric_name_string}"`. Por exemplo, a string do erro quadrático médio avaliado nos dados de validação deve ser `"val_mean_absolute_error"`.

* Envolva-a em `keras_tuner.Objective`. Normalmente, é necessário envolver o objetivo em um objeto `keras_tuner.Objective` para especificar a direção em que otimizar o objetivo. Por exemplo, se quisermos minimizar o erro quadrático médio, podemos usar `keras_tuner.Objective("val_mean_absolute_error", "min")`. A direção deve ser `"min"` ou `"max"`.

* Passe o agrupado para o sintonizador.

Você pode ver o seguinte exemplo de código básico.

In [48]:

def build_regressor(hp):
    model = tf.keras.Sequential(
        [
            layers.Dense(units=hp.Int("units", 32, 128, 32), activation="relu"),
            layers.Dense(units=1),
        ]
    )
    model.compile(
        optimizer="adam",
        loss="mean_squared_error",
        # O objetivo é uma das métricas.
        metrics=[tf.keras.metrics.MeanAbsoluteError()],
    )
    return model


tuner = keras_tuner.RandomSearch(
    hypermodel=build_regressor,
    # O nome e a direção do objetivo.
    objective=keras_tuner.Objective("val_mean_absolute_error", direction="min"),
    max_trials=3,
    overwrite=True,
    directory="my_dir",
    project_name="built_in_metrics",
)

tuner.search(
    x=np.random.rand(100, 10),
    y=np.random.rand(100, 1),
    validation_data=(np.random.rand(20, 10), np.random.rand(20, 1)),
)

tuner.results_summary()

Trial 3 Complete [00h 00m 01s]
val_mean_absolute_error: 0.8806929588317871

Best val_mean_absolute_error So Far: 0.21110916137695312
Total elapsed time: 00h 00m 02s
Results summary
Results in my_dir/built_in_metrics
Showing 10 best trials
Objective(name="val_mean_absolute_error", direction="min")

Trial 0 summary
Hyperparameters:
units: 64
Score: 0.21110916137695312

Trial 1 summary
Hyperparameters:
units: 96
Score: 0.43424248695373535

Trial 2 summary
Hyperparameters:
units: 32
Score: 0.8806929588317871


### Métrica personalizada como objetivo

Você pode implementar sua própria métrica e usá-la como objetivo de pesquisa de hiperparâmetro. Aqui, usamos o erro quadrático médio (MSE) como exemplo.

Primeiro, nós implementamos a métrica MSE por meio de uma subclasse de `keras.metrics.Metric`. Lembre-se de dar um nome à sua métrica usando o argumento `name` de `super().__init__()`, que será usado posteriormente.
> *Observação*: o MSE é, na verdade, uma métrica incorporada, que pode ser importado com `keras.metrics.MeanSquaredError`. Este é apenas um exemplo para mostrar como usar uma métrica personalizada como objetivo de pesquisa de hiperparâmetro.

Para obter mais informações sobre a implementação de métricas personalizadas, consulte [este tutorial](https://keras.io/api/metrics/#creating-custom-metrics). Se você quiser desejar uma métrica com uma assinatura de função diferente de `update_state(y_true, y_pred, sample_weight)`, você poderá substituir o método `train_step()` do seu modelo modelo seguindo [este tutorial](https://keras.io/guides/customizing_what_happens_in_fit/#going-lowerlevel).

In [49]:
class CustomMetric(tf.keras.metrics.Metric):
    def __init__(self, **kwargs):
        # Especifique o nome da métrica como "custom_metric".
        super().__init__(name="custom_metric", **kwargs)
        self.sum = self.add_weight(name="sum", initializer="zeros")
        self.count = self.add_weight(name="count", dtype="int32", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        values = tf.keras.ops.square(y_true - y_pred)
        count = tf.keras.ops.shape(y_true)[0]
        if sample_weight is not None:
            sample_weight = tf.keras.ops.cast(sample_weight, self.dtype)
            values *= sample_weight
            count *= sample_weight
        self.sum.assign_add(tf.keras.ops.sum(values))
        self.count.assign_add(count)

    def result(self):
        return self.sum / tf.keras.ops.cast(self.count, "float32")

    def reset_states(self):
        self.sum.assign(0)
        self.count.assign(0)


Execute a pesquisa com o objetivo personalizado.

In [50]:

def build_regressor(hp):
    model = tf.keras.Sequential(
        [
            layers.Dense(units=hp.Int("units", 32, 128, 32), activation="relu"),
            layers.Dense(units=1),
        ]
    )
    model.compile(
        optimizer="adam",
        loss="mean_squared_error",
        # Coloque a métrica personalizada nas métricas.
        metrics=[CustomMetric()],
    )
    return model


tuner = keras_tuner.RandomSearch(
    hypermodel=build_regressor,
    # Especifique o nome e a direção do objetivo.
    objective=keras_tuner.Objective("val_custom_metric", direction="min"),
    max_trials=3,
    overwrite=True,
    directory="my_dir",
    project_name="custom_metrics",
)

tuner.search(
    x=np.random.rand(100, 10),
    y=np.random.rand(100, 1),
    validation_data=(np.random.rand(20, 10), np.random.rand(20, 1)),
)

tuner.results_summary()

Trial 3 Complete [00h 00m 01s]
val_custom_metric: 0.11389829963445663

Best val_custom_metric So Far: 0.06787848472595215
Total elapsed time: 00h 00m 02s
Results summary
Results in my_dir/custom_metrics
Showing 10 best trials
Objective(name="val_custom_metric", direction="min")

Trial 1 summary
Hyperparameters:
units: 64
Score: 0.06787848472595215

Trial 2 summary
Hyperparameters:
units: 32
Score: 0.11389829963445663

Trial 0 summary
Hyperparameters:
units: 96
Score: 0.19134631752967834


Se o seu objetivo personalizado for difícil de ser colocado em uma métrica personalizada, você também pode avaliar o modelo por conta própria em `HyperModel.fit()` e retornar o valor do objetivo.

O valor do objetivo seria minimizado por padrão. Nesse caso, não é necessário especificar o `objetivo` ao inicializar o sintonizador.

No entanto, nesse caso, o valor da métrica não será rastreado nos logs do Keras, apenas nos logs do KerasTuner.
> Portanto, esses valores não seriam exibidos por nenhuma exibição do TensorBoard usando as métricas do Keras.

In [51]:

class HyperRegressor(keras_tuner.HyperModel):
    def build(self, hp):
        model = tf.keras.Sequential(
            [
                layers.Dense(units=hp.Int("units", 32, 128, 32), activation="relu"),
                layers.Dense(units=1),
            ]
        )
        model.compile(
            optimizer="adam",
            loss="mean_squared_error",
        )
        return model

    def fit(self, hp, model, x, y, validation_data, **kwargs):
        model.fit(x, y, **kwargs)
        x_val, y_val = validation_data
        y_pred = model.predict(x_val)
        # Retorna um único float para minimizar.
        return np.mean(np.abs(y_pred - y_val))


tuner = keras_tuner.RandomSearch(
    hypermodel=HyperRegressor(),
    # Não há objetivo a ser especificado.
    # O objetivo é o valor de retorno de `HyperModel.fit()`.
    max_trials=3,
    overwrite=True,
    directory="my_dir",
    project_name="custom_eval",
)
tuner.search(
    x=np.random.rand(100, 10),
    y=np.random.rand(100, 1),
    validation_data=(np.random.rand(20, 10), np.random.rand(20, 1)),
)

tuner.results_summary()

Trial 3 Complete [00h 00m 01s]
default_objective: 0.4981072251778859

Best default_objective So Far: 0.4981072251778859
Total elapsed time: 00h 00m 02s
Results summary
Results in my_dir/custom_eval
Showing 10 best trials
Objective(name="default_objective", direction="min")

Trial 2 summary
Hyperparameters:
units: 64
Score: 0.4981072251778859

Trial 1 summary
Hyperparameters:
units: 32
Score: 0.5630087179285445

Trial 0 summary
Hyperparameters:
units: 96
Score: 0.61802369301215


Se você tiver várias métricas para rastrear no KerasTuner, mas usar apenas uma delas como objetivo, poderá retornar um dicionário, cujas chaves são os nomes das métricas e os valores são os valores das métricas, por exemplo, retornar `{“metric_a”: 1.0, “metric_b”, 2.0}`.

Use uma das chaves como o nome do objetivo, por exemplo, `keras_tuner.Objective(“metric_a”, “min”)`.

In [52]:

class HyperRegressor(keras_tuner.HyperModel):
    def build(self, hp):
        model = tf.keras.Sequential(
            [
                layers.Dense(units=hp.Int("units", 32, 128, 32), activation="relu"),
                layers.Dense(units=1),
            ]
        )
        model.compile(
            optimizer="adam",
            loss="mean_squared_error",
        )
        return model

    def fit(self, hp, model, x, y, validation_data, **kwargs):
        model.fit(x, y, **kwargs)
        x_val, y_val = validation_data
        y_pred = model.predict(x_val)
        # Retorna um dicionário de métricas para o KerasTuner rastrear.
        return {
            "metric_a": -np.mean(np.abs(y_pred - y_val)),
            "metric_b": np.mean(np.square(y_pred - y_val)),
        }


tuner = keras_tuner.RandomSearch(
    hypermodel=HyperRegressor(),
    # O objetivo é uma das chaves.
    # Maximizar o MAE negativo, equivalente a minimizar o MAE.
    objective=keras_tuner.Objective("metric_a", "max"),
    max_trials=3,
    overwrite=True,
    directory="my_dir",
    project_name="custom_eval_dict",
)
tuner.search(
    x=np.random.rand(100, 10),
    y=np.random.rand(100, 1),
    validation_data=(np.random.rand(20, 10), np.random.rand(20, 1)),
)

tuner.results_summary()

Trial 3 Complete [00h 00m 01s]
metric_a: -0.3460912875026351

Best metric_a So Far: -0.2837509138006842
Total elapsed time: 00h 00m 02s
Results summary
Results in my_dir/custom_eval_dict
Showing 10 best trials
Objective(name="metric_a", direction="max")

Trial 0 summary
Hyperparameters:
units: 32
Score: -0.2837509138006842

Trial 2 summary
Hyperparameters:
units: 64
Score: -0.3460912875026351

Trial 1 summary
Hyperparameters:
units: 96
Score: -0.36974070530610603


## Ajuste os fluxos de trabalho de ponta a ponta

Em alguns casos, é difícil alinhar seu código em funções de compilação e treino. Você também pode manter o fluxo de trabalho de ponta a ponta em um único lugar substituindo `Tuner.run_trial()`, que lhe dá controle total de uma avaliação. Você pode vê-lo como um otimizador de caixa preta para qualquer coisa.

### Ajuste qualquer função

Por exemplo, você pode encontrar um valor de `x` que minimize `f(x)=x*x+1`. No código a seguir, apenas definimos `x` como um hiperparâmetro e retornamos `f(x)` como o valor objetivo. Os argumentos `hypermodel` e `objective` para inicializar o sintonizador podem ser omitidos.

In [53]:

class MyTuner(keras_tuner.RandomSearch):
    def run_trial(self, trial, *args, **kwargs):
        # Obter o hp do teste.
        hp = trial.hyperparameters
        # Defina “x” como um hiperparâmetro.
        x = hp.Float("x", min_value=-1.0, max_value=1.0)
        # Retorna o valor objetivo a ser minimizado.
        return x * x + 1


tuner = MyTuner(
    # Nenhum hipermodelo ou objetivo especificado.
    max_trials=20,
    overwrite=True,
    directory="my_dir",
    project_name="tune_anything",
)

# Não há necessidade de passar nada para search()
# a menos que você os use em run_trial().
tuner.search()
print(tuner.get_best_hyperparameters()[0].get("x"))

Trial 20 Complete [00h 00m 00s]
default_objective: 1.65720670166111

Best default_objective So Far: 1.0036180512809396
Total elapsed time: 00h 00m 00s
0.060150239242579895


### Mantenha o código do Keras separado

Você pode manter todo o seu código do Keras inalterado e usar o KerasTuner para ajustá-lo. Isso é útil se você não puder modificar o código do Keras por algum motivo.

Isso também lhe dá mais flexibilidade. Você não precisa separar a construção do modelo e o código de treinamento. No entanto, esse fluxo de trabalho não ajudaria você a salvar o modelo ou conectar-se aos plug-ins do TensorBoard.

Para salvar o modelo, você pode usar `trial.trial_id`, que é uma cadeia de caracteres para identificar uma tentativa, para construir caminhos diferentes para salvar os modelos de diferentes tentativas.

In [54]:
import os


def keras_code(units, optimizer, saving_path):
    # Construir modelo
    model = tf.keras.Sequential(
        [
            layers.Dense(units=units, activation="relu"),
            layers.Dense(units=1),
        ]
    )
    model.compile(
        optimizer=optimizer,
        loss="mean_squared_error",
    )

    # Preparar dados
    x_train = np.random.rand(100, 10)
    y_train = np.random.rand(100, 1)
    x_val = np.random.rand(20, 10)
    y_val = np.random.rand(20, 1)

    # Treinar e avaliar o modelo
    model.fit(x_train, y_train)

    # Salvar o modelo
    model.save(saving_path)

    # Retorna um único float como o valor objetivo.
    # Você também pode retornar um dicionário
    # de {metric_name: metric_value}.
    y_pred = model.predict(x_val)
    return np.mean(np.abs(y_pred - y_val))


class MyTuner(keras_tuner.RandomSearch):
    def run_trial(self, trial, **kwargs):
        hp = trial.hyperparameters
        return keras_code(
            units=hp.Int("units", 32, 128, 32),
            optimizer=hp.Choice("optimizer", ["adam", "adadelta"]),
            saving_path=os.path.join("/tmp", f"{trial.trial_id}.keras"),
        )


tuner = MyTuner(
    max_trials=3,
    overwrite=True,
    directory="my_dir",
    project_name="keep_code_separate",
)
tuner.search()
# Retreinando o modelo
best_hp = tuner.get_best_hyperparameters()[0]
keras_code(**best_hp.values, saving_path="/tmp/best_model.keras")

Trial 3 Complete [00h 00m 00s]
default_objective: 0.3756241505326371

Best default_objective So Far: 0.3756241505326371
Total elapsed time: 00h 00m 01s
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5641  
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


0.4610180642441408

## O KerasTuner inclui aplicativos ajustáveis pré-fabricados: HyperResNet e HyperXception

Esses são hipermodelos prontos para uso para visão computacional.

Eles vêm pré-compilados com `loss=“categorical_crossentropy”` e
`metrics=[“accuracy”]`.

In [55]:
from keras_tuner.applications import HyperResNet

hypermodel = HyperResNet(input_shape=(28, 28, 1), classes=10)

tuner = keras_tuner.RandomSearch(
    hypermodel,
    objective="val_accuracy",
    max_trials=2,
    overwrite=True,
    directory="my_dir",
    project_name="built_in_hypermodel",
)